<a href="https://colab.research.google.com/github/Sarah-wookey/RL_NetHack_2020/blob/main/Actor_Critic_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# install prerequisites for nle
!sudo apt-get install -y build-essential autoconf libtool pkg-config \
    python3-dev python3-pip python3-numpy git libncurses5-dev \
    libzmq3-dev flex bison

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
python3-numpy is already the newest version (1:1.13.3-2ubuntu1).
python3-numpy set to manually installed.
git is already the newest version (1:2.17.1-1ubuntu0.7).
libncurses5-dev is already the newest version (6.1-1ubuntu1.18.04).
python3-dev is already the newest version (3.6.7-1~18.04).
libzmq3-dev is already the newest version (4.2.5-1ubuntu0.2).
The following additional packages will be installed:
  automake autotools-dev file libbison-dev libfl-dev libfl2 libmagic-mgc
  libmagic1 libsigsegv2 m4 python-pip-whl python3-asn1crypto
  python3-cffi-backend python3-crypto python3-cryptography python3-idna
  python3-keyring python3-keyrings.alt python3-pkg-resources
  python3-secretstorage python3-setuptools python3-six python3-wheel
  python3-xdg
Suggested packages:
  autoco

In [ ]:
# download, build and install flatbuffers

!git clone https://github.com/google/flatbuffers.git
# all these commands have to be run in the same directory and !cd doesn't change
# the directory permanently in colab see: 
# https://stackoverflow.com/questions/48298146/changing-directory-in-google-colab-breaking-out-of-the-python-interpreter
!cd flatbuffers && cmake -G "Unix Makefiles" && make && sudo make install

Cloning into 'flatbuffers'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 19405 (delta 8), reused 14 (delta 1), pack-reused 19369
Receiving objects: 100% (19405/19405), 11.82 MiB | 23.88 MiB/s, done.
Resolving deltas: 100% (13474/13474), done.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for strtof_l
-- Looking for strtof_l - found
--

In [ ]:
# the next step requires a version of cmake > 3.14.0
!pip install cmake==3.15.3

     |████████████████████████████████| 16.5MB 187kB/s 
  Found existing installation: cmake 3.12.0
    Uninstalling cmake-3.12.0:
      Successfully uninstalled cmake-3.12.0


In [ ]:
# add -v for verbose if there are any errors
!pip install nle 

     |████████████████████████████████| 6.7MB 5.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/00/84/fc9dc13ee536ba5e6b8fd10ce368fea5b738fe394c3b296cde7c9b144a92/pybind11-2.6.1-py2.py3-none-any.whl
  Created wheel for nle: filename=nle-0.6.0-cp36-cp36m-linux_x86_64.whl size=2847878 sha256=918dd39743ba30a5c1bc08aadf26e0a8774b4b77639a1545171ea70190012126
  Stored in directory: /root/.cache/pip/wheels/b5/4d/7c/e4c74b776f945ec1bc9bf01dc94bc226e452cf7dd2aba347a2
Successfully built nle


In [ ]:
!pip install "nle[agent]"

In [ ]:
!pip install pyvirtualdisplay

# Imports

In [ ]:
import os
import matplotlib as mp
# if os.environ.get('DISPLAY','') == '':
#     print('no display found. Using non-interactive Agg backend')
#     mp.use('Agg')
import matplotlib.pyplot as plt

import numpy as np
import gym
import nle
import random
from gym import spaces
from nle import nethack
from collections import deque
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.distributions import Categorical
import random


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cwd = os.getcwd()

In [ ]:
Colab = False

if Colab:
    # Imports specifically for google colab/drive
    from google.colab import files, drive
    drive.mount('/content/gdrive')
    dir = '/content/gdrive/My Drive/Colab Notebooks/Nethack-AC/'
    os.makedirs(dir, exist_ok=True)
else:
    dir = './'

path = lambda fname: os.path.join(dir, fname)


# ..

In [ ]:


# convert the observation from env to the -> glyphs_matrix,around_agent,agent_stat
def format_observations(observation, keys=("glyphs", "blstats")):
    observations = {}
    for key in keys:
        entry = observation[key]
        entry = torch.from_numpy(entry)
        entry = entry.view((1, 1) + entry.shape)  # (...) -> (T,B,...).
        observations[key] = entry
    return observations




# Agents

In [ ]:
class AbstractAgent:
    """
    AbstractAgent
    """
    def __init__(self):
        raise NotImplementedError()

    def act(self, observation):
        raise NotImplementedError()


class MyAgent(AbstractAgent):
    def __init__(self, observation_space, action_space):
        self.observation_space = observation_space
        self.action_space = action_space
        # TODO Initialise your agent's models
        self.model = PolicyValueNetwork(action_space).to(device)

        # for example, if your agent had a Pytorch model it must be load here
        PATH = cwd+'/models/Nethackac.pth'
        self.model.load_state_dict(torch.load(PATH, map_location=torch.device(device)))

    def act(self, observation):
        # Perform processing to observation

        # TODO: return selected action
        state = format_observations(observation)
 

        with torch.no_grad():
            dist, value = self.model(state)
            action = dist.sample().item()
        return action.item()

class RandomAgent(AbstractAgent):
    def __init__(self, observation_space, action_space):
        self.observation_space = observation_space
        self.action_space = action_space

    def act(self, observation):
        return self.action_space.sample()


# Network

In [ ]:
def run_episode(env):
    # create instance of MyAgent
    # from MyAgent import MyAgent
    agent = MyAgent(env.observation_space, env.action_space.n)

    done = False
    episode_return = 0.0
    state = env.reset()

    while not done:
        # pass state to agent and let agent decide action
        action = agent.act(state)
        new_state, reward, done, _ = env.step(action)
        episode_return += reward
        state = new_state
    return episode_return

In [ ]:
class PolicyValueNetwork(nn.Module):

    def __init__(
        self,
        observation_shape,
        num_actions,
        use_lstm = True,
        embedding_dim=32,
        crop_dim=9,
        num_layers=5):


        super(PolicyValueNetwork, self).__init__()

        self.glyph_shape = observation_shape["glyphs"].shape
        self.blstats_size = observation_shape["blstats"].shape[0]

        self.num_actions = num_actions
        self.use_lstm = use_lstm

        self.H = self.glyph_shape[0]
        self.W = self.glyph_shape[1]

        self.k_dim = embedding_dim
        self.h_dim = 512

        self.crop_dim = crop_dim

        self.crop = Crop(self.H, self.W, self.crop_dim, self.crop_dim)

        self.embed = nn.Embedding(nethack.MAX_GLYPH, self.k_dim)

        K = embedding_dim  # number of input filters
        F = 3  # filter dimensions
        S = 1  # stride
        P = 1  # padding
        M = 16  # number of intermediate filters
        Y = 8  # number of output filters
        L = num_layers  # number of convnet layers

        in_channels = [K] + [M] * (L - 1)
        out_channels = [M] * (L - 1) + [Y]

        def interleave(xs, ys):
            return [val for pair in zip(xs, ys) for val in pair]

        conv_extract = [
            nn.Conv2d(
                in_channels=in_channels[i],
                out_channels=out_channels[i],
                kernel_size=(F, F),
                stride=S,
                padding=P,
            )
            for i in range(L)
        ]

        self.extract_representation = nn.Sequential(
            *interleave(conv_extract, [nn.ELU()] * len(conv_extract))
        )

        # CNN crop model.
        conv_extract_crop = [
            nn.Conv2d(
                in_channels=in_channels[i],
                out_channels=out_channels[i],
                kernel_size=(F, F),
                stride=S,
                padding=P,
            )
            for i in range(L)
        ]

        self.extract_crop_representation = nn.Sequential(
            *interleave(conv_extract_crop, [nn.ELU()] * len(conv_extract))
        )

        out_dim = self.k_dim
        # CNN over full glyph map
        out_dim += self.H * self.W * Y

        # CNN crop model.
        out_dim += self.crop_dim ** 2 * Y

        self.embed_blstats = nn.Sequential(
            nn.Linear(self.blstats_size, self.k_dim),
            nn.ReLU(),
            nn.Linear(self.k_dim, self.k_dim),
            nn.ReLU(),
        )

        self.fc = nn.Sequential(
            nn.Linear(out_dim, self.h_dim),
            nn.ReLU(),
            nn.Linear(self.h_dim, self.h_dim),
            nn.ReLU(),
        )

        if self.use_lstm:
            self.core = nn.LSTM(self.h_dim, self.h_dim, num_layers=1)

        self.policy = nn.Linear(self.h_dim, self.num_actions)
        self.state_value = nn.Linear(self.h_dim, 1)


    def _select(self, embed, x):
        # Work around slow backward pass of nn.Embedding, see
        # https://github.com/pytorch/pytorch/issues/24912
        out = embed.weight.index_select(0, x.reshape(-1))
        return out.reshape(x.shape + (-1,))

    def initial_state(self, batch_size=1):
        if not self.use_lstm:
            return tuple()
        return tuple((
            torch.zeros(self.core.num_layers, batch_size, self.core.hidden_size),
            torch.zeros(self.core.num_layers, batch_size, self.core.hidden_size)
        ))

    
    def forward(self, state):

        glyphs = state["glyphs"]
        # -- [T x B x F]
        blstats = state["blstats"]
        T, B, *_ = glyphs.shape
        # -- [B' x H x W]
        glyphs = torch.flatten(glyphs, 0, 1)  # Merge time and batch.
        # -- [B' x F]
        blstats = blstats.view(T * B, -1).float()
        # -- [B x H x W]
        glyphs = glyphs.long()
        # -- [B x 2] x,y coordinates
        coordinates = blstats[:, :2]
        # TODO ???
        # coordinates[:, 0].add_(-1)
        # -- [B x F]
        # FIXME: hack to use compatible blstats to before
        # blstats = blstats[:, [0, 1, 21, 10, 11]]
        blstats = blstats.view(T * B, -1).float()
        # -- [B x K]
        blstats_emb = self.embed_blstats(blstats)
        assert blstats_emb.shape[0] == T * B
        reps = [blstats_emb] # representations
        # -- [B x H' x W']
        crop = self.crop(glyphs, coordinates)
        # print("crop", crop)
        # print("at_xy", glyphs[:, coordinates[:, 1].long(), coordinates[:, 0].long()])
        ## self.extract_crop_representation and self.extract_representation forward pass the same
        # -- [B x H' x W' x K]
        crop_emb = self._select(self.embed, crop)
        # CNN crop model.
        # -- [B x K x W' x H']
        crop_emb = crop_emb.transpose(1, 3)  # -- TODO: slow?
        # -- [B x W' x H' x K]
        crop_rep = self.extract_crop_representation(crop_emb)
        # -- [B x K']
        crop_rep = crop_rep.view(T * B, -1)
        assert crop_rep.shape[0] == T * B
        reps.append(crop_rep)
        # -- [B x H x W x K]
        glyphs_emb = self._select(self.embed, glyphs)
        # glyphs_emb = self.embed(glyphs)
        # -- [B x K x W x H]
        glyphs_emb = glyphs_emb.transpose(1, 3)  # -- TODO: slow?
        # -- [B x W x H x K]
        glyphs_rep = self.extract_representation(glyphs_emb)
        # -- [B x K']
        glyphs_rep = glyphs_rep.view(T * B, -1)
        assert glyphs_rep.shape[0] == T * B
        # -- [B x K'']
        reps.append(glyphs_rep)
        st = torch.cat(reps, dim=1)
        # -- [B x K]
        st = self.fc(st) 
        # -- [B x A]
        policy_logits = self.policy(st)
        # -- [B x A]
        # print(F.softmax(policy_logits, dim=1))
        # if self.training:
        #     action = torch.multinomial(F.softmax(policy_logits, dim=1), num_samples=1)
        # else:
        #     # Don't sample when testing.
        #     action = torch.argmax(policy_logits, dim=1)


        state_value = self.state_value(st)
        prob = F.softmax(policy_logits,dim=-1)
        dist = Categorical(prob)
        return dist, state_value   

In [ ]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns


# Crop

In [ ]:

class Crop(nn.Module):
    """Helper class for NetHackNet below."""

    def __init__(self, height, width, height_target, width_target):
        super(Crop, self).__init__()
        self.width = width
        self.height = height
        self.width_target = width_target
        self.height_target = height_target
        width_grid = _step_to_range(2 / (self.width - 1), self.width_target)[
            None, :
        ].expand(self.height_target, -1)
        height_grid = _step_to_range(2 / (self.height - 1), height_target)[
            :, None
        ].expand(-1, self.width_target)

        # "clone" necessary, https://github.com/pytorch/pytorch/issues/34880
        self.register_buffer("width_grid", width_grid.clone())
        self.register_buffer("height_grid", height_grid.clone())

    def forward(self, inputs, coordinates):
        """Calculates centered crop around given x,y coordinates.
        Args:
           inputs [B x H x W]
           coordinates [B x 2] x,y coordinates
        Returns:
           [B x H' x W'] inputs cropped and centered around x,y coordinates.
        """
        assert inputs.shape[1] == self.height, f'{ inputs.shape[1],self.height}'
        assert inputs.shape[2] == self.width

        inputs = inputs[:, None, :, :].float()

        x = coordinates[:, 0]
        y = coordinates[:, 1]

        x_shift = 2 / (self.width - 1) * (x.float() - self.width // 2)
        y_shift = 2 / (self.height - 1) * (y.float() - self.height // 2)

        grid = torch.stack(
            [
                self.width_grid[None, :, :] + x_shift[:, None, None],
                self.height_grid[None, :, :] + y_shift[:, None, None],
            ],
            dim=3,
        )

        # TODO: only cast to int if original tensor was int
        return (
            torch.round(F.grid_sample(inputs, grid, align_corners=True))
            .squeeze(1)
            .long()
        )

# Run

In [ ]:
def save_checkpoint(
    fname,
    policy_model,
    optimizer,
    current_step_number,
    scores,
    train_loss,
    download=False
):

    checkpoint = {
        'current_step_number': current_step_number,
        'scores': scores,
        'train_loss': train_loss,
        'model_state_dict': policy_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()}

    torch.save(checkpoint, path(fname))
    if download and Colab: files.download(path) # not working well 

In [ ]:
def dict_to_device(dic):
    for key, value in dic.items():
        dic[key] = dic[key].to(device)
    return dic

In [ ]:

def train_ac(env, policy_model,flags, seed, learning_rate,
                                             number_episodes,
                                             gamma, verbose=True):
    # set random seeds (for reproducibility)
    max_steps=1e4
    num_step_td_update = 5
    torch.cuda.manual_seed_all(flags['seed'])
    np.random.seed(flags['seed'])
    env.seed(flags['seed'])
    random.seed(flags['seed'])
    optimizer = optim.RMSprop(policy_model.parameters(),lr=learning_rate,eps=0.000001)
    scores = [0.0]
    scores_mean = []
    current_step_number = 0
    train_loss = [0]
    state = env.reset()
    
    while current_step_number < max_steps:
        state_value = []
        rewards = []
        log_prob_actions = []
        masks = []

        for _ in range(num_step_td_update):
            
            
            state_input=format_observations(state)
            state_input=dict_to_device(state_input)
            dist,value = policy_model(state_input)
            action = dist.sample()

            log_p = dist.log_prob(action)
            state_prime,reward,done,_ = env.step(action)
            
            scores[-1]+=reward

            # clip rewards
            

            log_prob_actions.append(log_p)
            state_value.append(value)
            masks.append(1 - done)
            rewards.append(reward)
            
            current_step_number += 1
            state = state_prime

            if (len(scores) % 20 == 0 and done):
                save_checkpoint(
                    path('AC_1'+f'-{len(scores)}.pt'),
                    policy_model, optimizer, current_step_number,
                    scores, train_loss)


            if done:
                mean_100ep_reward = round(np.mean(scores[-25:-1]), 1)
                mean_100ep_loss = round(np.mean((train_loss)[-25:-1]), 1)
                print(
                    "steps: {}".format(current_step_number),
                    "episodes: {}".format(len(scores)),
                    "ep reward: {}".format(scores[-1]),
                    "last loss: {}".format((train_loss[-1])),
                    (f"mean_reward: {mean_100ep_reward:.4f}"),
                    (f"mean_loss: {mean_100ep_loss:.7f}")
                )
                            
                if len(scores)%100==0:
                    scores_mean.append(round(np.mean(scores[-100:-1]), 1))
                state = env.reset()
                scores.append(0.0)

        state_input=format_observations(state)
        state_input=dict_to_device(state_input)
        dist,value = policy_model(state_input)
        action = dist.sample()

        _,next_value = policy_model(state_input)
        
        returns = compute_returns(next_value, rewards, masks,gamma)
        
        state_value = torch.cat(state_value)
        log_prob_actions = torch.cat(log_prob_actions)
        returns   = torch.cat(returns).detach()

        change = returns - state_value
        p_loss = -torch.mean(log_prob_actions*change.detach())
        v_loss = 0.5*torch.mean(torch.pow(change,2))
        loss = p_loss + v_loss

        optimizer.zero_grad()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(policy_model.parameters(), 40)
        optimizer.step()

        train_loss.append(loss.item())

    PATH = cwd+'/models/Nethackac.pth'
    torch.save(policy_model.state_dict(), PATH) 
    
    return policy_model,PATH, scores.copy(),scores_mean.copy()


In [ ]:
def _step_to_range(delta, num_steps):
    """Range of `num_steps` integers with distance `delta` centered around zero."""
    return delta * torch.arange(-num_steps // 2, num_steps // 2)

# Main

In [ ]:
def main():
    # Seed
    seeds = [1,2,3,4,5]
    
    # Initialise environment
    env = gym.make("NetHackScore-v0",observation_keys=("glyphs", "blstats"))
    # hyper-parameters
    gamma = 0.99
    learning_rate = 0.02
    # seed = 214
    seed = np.random.choice(seeds)
    torch.manual_seed(seed)
    env.seed(seed)
    number_episodes = 1000
    hyper_params = {
        "learning_rate": 0.00048,  # learning rate for RMSprob
        "discount-factor": 0.99,  # discount factor
        "num-steps": int(1e6),  # total number of steps to run the environment for
        "batch-size": 256,  # number of transitions to optimize at the same time
        "learning-starts": 10000,  # number of steps before learning starts
        "learning-freq": 5,  # number of iterations between every optimization step
        "use-double-dqn": True,  # use double deep Q-learning
        "target-update-freq": 1000,  # number of iterations between every target network update
        "eps-start": 1.0,  # e-greedy start threshold
        "eps-end": 0.01,  # e-greedy end threshold
        "eps-fraction": .5,  # fraction of num-steps
        "print-freq":1,
        "save-freq":25,
        "save-name":'dqn_1',
        "seed":seed
    }

    policy_model = PolicyValueNetwork(env.observation_space,env.action_space.n,False).to(device)
    net,path, scores,scores_mean = train_ac(env, policy_model,hyper_params, seed, learning_rate,
                                             number_episodes,
                                             gamma, verbose=True)
    
    #Number of times each seed will be run
    num_runs = 10

    #Run a few episodes on each seed
    rewards = []
    for seed in seeds:
        env.seed(seed)
        seed_rewards = []
        for i in range(num_runs):
            seed_rewards.append(run_episode(env))
        rewards.append(np.mean(seed_rewards))

    # Close environment and print average reward
    env.close()
#     print("Average Reward: %f" %(np.mean(rewards)))
    
    
    plt.figure(figsize=(15,15))
    plt.plot(rewards)
    plt.ylabel("reward")
    plt.xlabel("episode")
    plt.title("episode rewards testing")
    plt.savefig(cwd+'/img/actor_critic_testing_reward_nethackac.png')
    
    
    plt.figure(figsize=(15,15))
    plt.plot(scores)
    plt.ylabel("score")
    plt.xlabel("episode")
    plt.savefig(cwd+'/img/actor_critic_traing_reward_nethackac.png')
    
    plt.figure(figsize=(15,15))
    plt.plot(scores_mean)
    plt.ylabel("score(average over 100 eps)")
    plt.xlabel("episode")
    plt.savefig(cwd+'/img/actor_critic_traing_reward_nethackac.png')

# ...

In [ ]:
if __name__ == '__main__':
    main()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


steps: 5000 episodes: 1 ep reward: -49.93999999999863 last loss: 4.3204535357271966e+19 mean_reward: nan mean_loss: 43204535357271965696.0000000
steps: 10000 episodes: 2 ep reward: -49.98999999999862 last loss: 2.835441256527351e+19 mean_reward: -49.9000 mean_loss: 28354412565273509888.0000000


FileNotFoundError: ignored

# .,

In [ ]:


plt.figure(figsize=(15,15))
plt.plot(rewards)
plt.ylabel("reward")
plt.xlabel("episode")
plt.title("episode rewards testing")
plt.savefig(cwd+'/img/actor_critic_testing_reward_nethackac.png')


plt.figure(figsize=(15,15))
plt.plot(scores)
plt.ylabel("score")
plt.xlabel("episode")
plt.savefig(cwd+'/img/actor_critic_traing_reward_nethackac.png')

plt.figure(figsize=(15,15))
plt.plot(scores_mean)
plt.ylabel("score(average over 100 eps)")
plt.xlabel("episode")
plt.savefig(cwd+'/img/actor_critic_traing_reward_nethackac.png')